In [1]:
import os
import pandas as pd
import numpy as np
import csv
from scipy.sparse import coo_matrix
import time

In [2]:
path_dir = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\data'
excel_file = 'shenghuo_comments_32741.xlsx'
path_file = os.path.join(path_dir,excel_file)

In [3]:
filter_df = pd.read_excel(path_file,usecols=['text','user_name','create_time','clean_text','cut_word','pos_word'])
filter_df.index = (filter_df.reset_index()).index + 1

In [4]:
word = []
for line in filter_df['cut_word']:
    line = line.split(' ')
    for w in line:
        if len(w) > 1:
            if w not in word:
                word.append(w)

In [7]:
len(word)

19975

In [10]:
def co_word_vector(word,line_lists):
    word_vector = np.zeros([len(word),len(word)], dtype='float16')
    #word_vector = coo_matrix((len(word),len(word)), dtype=np.int8).toarray()
    print(f"矩阵形状为：{word_vector.shape}")
    print(f"不重复词数为：{len(word)}")
    print(f"评论总数为：{len(line_lists)}")
    m = 1
    for line in line_lists:

        #去除读取行中的单字符。
        line_list = line.split(' ')
        nums = []
        for w in line_list:
            if len(w) > 1:
                nums.append(w)

        #循环遍历关键词所在位置 设置word_vector计数
        i = 0
        j = 0
        while i<len(nums):         #ABCD共现 AB AC AD BC BD CD加1
            j = i + 1
            w1 = nums[i]           #第一个单词
            while j<len(nums):
                w2 = nums[j]       #第二个单词
                #从word数组中找到单词对应的下标
                k = 0
                n1 = 0
                while k<len(word):
                    if w1==word[k]: #如果是自身与自身，则不做任何操作。
                        n1 = k
                        break
                    k = k +1
                #寻找第二个关键字位置
                k = 0
                n2 = 0
                while k<len(word):
                    if w2==word[k]:
                        n2 = k
                        break
                    k = k +1
                #重点: 词频矩阵赋值 只计算上三角
                if n1<=n2:
                    word_vector[n1][n2] = word_vector[n1][n2] + 1
                else:
                    word_vector[n2][n1] = word_vector[n2][n1] + 1
                #print n1, n2, w1, w2
                j = j + 1
            i = i + 1
        if m == 10000 or m == 20000 or m == 30000:
            print(f'已处理{m}条评论，请稍等。')
        m += 1
    return word_vector
def save_co_word_to_txt(word,word_vector,txt_name):
    res = open(path_dir+'\\'+txt_name, "a+", encoding='utf-8')
    i = 0
    while i<len(word):
        w1 = word[i]
        j = 0
        while j<len(word):
            w2 = word[j]
            #判断两个词是否共现 共现&词频不为0的写入文件
            if word_vector[i][j]>0:
                #print w1 +" " + w2 + " "+ str(int(word_vector[i][j]))
                res.write(w1 +" " + w2 + " "+ str(int(word_vector[i][j]))  +  "\n")
            j = j + 1
        i = i + 1
    res.close()
def save_co_word_to_csv(word,word_vector,csv_name):
    c = open(path_dir+'\\'+csv_name,"w", encoding='utf-8', newline='')    #解决空行
    #c.write(codecs.BOM_UTF8)                                 #防止乱码
    writer = csv.writer(c)                                    #写入对象
    writer.writerow(['Word1', 'Word2', 'Weight'])

    i = 0
    while i<len(word):
        w1 = word[i]
        j = 0
        while j<len(word):
            w2 = word[j]
            #判断两个词是否共现 共现词频不为0的写入文件
            if word_vector[i][j]>0:
                #写入文件
                templist = []
                templist.append(w1)
                templist.append(w2)
                templist.append(str(int(word_vector[i][j])))
                #print templist
                writer.writerow(templist)
            j = j + 1
        i = i + 1
    c.close()

In [11]:
word_vector1 = co_word_vector(word,filter_df['cut_word'])
print('共现矩阵构建完毕。')
print(time.time())
save_co_word_to_txt(word,word_vector1,'co_word.txt')
print('写入txt文件完毕。')
print(time.time())
save_co_word_to_csv(word,word_vector1,'co_word.csv')
print('写入csv文件完毕。')
print(time.time())

(19975, 19975)
19975
32741


KeyboardInterrupt: 